In [1]:
import pandas as pd


df = pd.read_csv("./dataset/dataset.csv")
df_check = pd.read_csv("./dataset/check.csv")
y = df.pop("smoking").astype(int)
X = df

y_check = df_check.pop("smoking").astype(int)
X_check = df_check



In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

rf_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(
        n_estimators=166,
        min_samples_leaf=2,
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=123,
        max_depth=19,
        min_samples_split=5,
        bootstrap=False
    ))
])

rf_pipe.fit(X,y)

proba = rf_pipe.predict_proba(X_check)[:, 1]
pred  = (proba >= 0.5).astype(int)

print(classification_report(y_check, pred, digits=3))

              precision    recall  f1-score   support

           0      0.804     0.871     0.836      3164
           1      0.741     0.633     0.683      1836

    accuracy                          0.784      5000
   macro avg      0.772     0.752     0.760      5000
weighted avg      0.781     0.784     0.780      5000



In [6]:

rf = rf_pipe.named_steps["rf"]
import pandas as pd
imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop features:\n", imp)


Top features:
 Gtp                    0.092388
triglyceride           0.079111
hemoglobin             0.067967
LDL                    0.061018
HDL                    0.058364
waist(cm)              0.058306
Cholesterol            0.058223
ALT                    0.057161
systolic               0.056757
fasting blood sugar    0.055094
AST                    0.051047
relaxation             0.050196
age                    0.041002
height(cm)             0.035922
serum creatinine       0.033781
weight(kg)             0.033140
eyesight(left)         0.032457
eyesight(right)        0.031912
gender                 0.013705
tartar                 0.013345
dental caries          0.009720
Urine protein          0.004899
hearing(right)         0.002362
hearing(left)          0.002124
dtype: float64


In [ ]:
# upsampling auf Female / Female + smokers

# RandomForestClassifier, Support Vector Machine, MLP (Multi Layer)

# Hyperparameter, Gridsearch

# Correlation, Signifikanztests

# Literaturrecherche (verweis auf die Correlations, etc.)

# 